In [1]:
import eunomia
from eunomia.agents import Eunomia
from eunomia.tools import EunomiaTools

In [2]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import warnings

# Disable all warning messages
warnings.filterwarnings("ignore")

## Obtaining Water Stability Information from a MOF Paper

In [3]:
paper_id = 69
paper_file =f'../Data/chosen_papers/{paper_id}.pdf'
docs_processor = eunomia.LoadDoc(file_name=paper_file, encoding="utf8")
sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments', 'references\n'],
                                             chunk_size=1000, chunk_overlap=20, chunking_type='fixed-size')

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

Embedding_model = 'text-embedding-ada-002' 
faiss_index_path = f"../Data/chosen_papers/faiss/{Embedding_model}/faiss_index_{Embedding_model}_{paper_id}.pkl"
faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))

In [5]:
tools = EunomiaTools(tool_names = ['read_doc', 'eval_justification', 'recheck_justification'], vectorstore=faiss_index).get_tools()
agent = Eunomia(tools=tools, model='gpt-4', get_cost=True)
agent.run(prompt="Read the document and find the MOFS and their water stability.\
            \Check your answers and if the reasoning is not valid, try to find better reasons.")



> Entering new AgentExecutor chain...
I need to read the document first to find information about the water stability of MOFs.
Action: read_doc
Action Input: User's original promptThis model's maximum context length is 8192 tokens. However, your messages resulted in 9134 tokens. Please reduce the length of the messages.

k=9 results hitting the token limit for the chosen LLM. Reducing k and retrying...

This model's maximum context length is 8192 tokens. However, your messages resulted in 8459 tokens. Please reduce the length of the messages.

k=8 results hitting the token limit for the chosen LLM. Reducing k and retrying...


Observation: 1. MOF: [ZrO(O2C-R-CO2)] (R =C6H4(MIL-140A), C10H6(B), C12H8(C), C12N2H6Cl2(D))
   Water Stability: Stable
   Probability Score: 0.9
   Justification: "The MIL-140 solids crystallize in the space group(SG) C2/c(No. 15) (MIL-140A and C) or Cc (No. 9) (MIL-140B and D; Supporting Information, Table S3). These solids all have the same cparameter (ca. 7

'The MOFs [ZrO(O2C-R-CO2)] and UiO-66(Zr) are stable in water. The justification for the water stability of [ZrO(O2C-R-CO2)] is that it exhibits almost no loss of free water and can be considered more hydrophobic than the UiO-66 counterpart. The justification for the water stability of UiO-66(Zr) is that it remains unaltered upon water adsorption/desorption cycles.'